In [1]:
# install msodbcsql17
%%sh
apt-get install -y unixodbc-dev
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
unixodbc-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main armhf Packages [15.1 kB]
Hi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   2249      0 --:--:-- --:--:-- --:--:--  2249
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    259      0 --:--:-- --:--:-- --:--:--   258
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide

In [2]:
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 5.5 MB/s eta 0:00:00


In [3]:
import pyodbc
import pandas as pd
import os

server=''
user=''
password=''

#conn=pymssql.connect(server=server, user=user, password=password, port=1433)
conn=pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+server+';'
    'UID='+user+';'
    'PWD='+password+';'
)
print(conn)

In [4]:
def query_data(fac_no, code_no):
    # query data
    qstr="""SELECT fac_no, AppType, DType, AppTime, ProduceCode, app_qty
            FROM IMP_3.dbo.Waste_tmp_month_statistics wtms
            WHERE fac_no = '{}'
                AND CodeNo = '{}'
            ORDER BY AppTime"""
    df1=pd.read_sql(qstr.format(fac_no, code_no), conn)

    qstr="""SELECT ControlNo, AccName, VocCode1, Square1, SquareUnit1, Square2, SquareUnit2, Square3, SquareUnit3, VocCode1Name
            FROM IMP.dbo.Industry i --事業基本資料
            WHERE ControlNo = '{}'"""
    df2=pd.read_sql(qstr.format(fac_no), conn)

    voc_no=df2['VocCode1'][0] # 行業別, e.g. '5620'
    produce_code=df1['ProduceCode'][0] # 製程代碼, e.g. '560001'

    qstr="""SELECT '{}', Waste_name, ProduceName
            FROM IMP_3.dbo.Waste_View_Waste wvw --廢棄物八碼組合資料檔
            WHERE Waste_no = '{}'
                AND voc_no = '{}'
                AND ProduceCode = '{}'"""
    df3=pd.read_sql(qstr.format(fac_no, code_no, voc_no, produce_code), conn) # garbled text because the type declared in DB is 'varchar' instead of 'nvarchar'
    if df3.empty:
        df3=pd.DataFrame({'fac_no': [fac_no], 'Waste_name': [None], 'ProduceName': [None]})
    else:
        df3=df3.iloc[[0]]
        df3=df3.rename(columns={'': 'fac_no'})

    qstr="""SELECT fac_no, Pd_qty_mMax, Pd_qty_mAvg
            FROM IMP_3.dbo.Waste_Tmp_ClePlanPro wtcpp
            WHERE fac_no = '{}'
                AND CodeNo = '{}'"""
    df4=pd.read_sql(qstr.format(fac_no, code_no), conn)

    # merge data
    # df1: fac_no, AppType, DType, AppTime, ProduceCode, app_qty
    # df2: ControlNo, AccName, VocCode1, Square1, SquareUnit1, Square2, SquareUnit2, Square3, SquareUnit3, VocCode1Name
    # df3: fac_no, Waste_name, ProduceName
    # df4: fac_no, Pd_qty_mMax, Pd_qty_mAvg
    df_combine=df1.merge(df2, how='left', left_on='fac_no', right_on='ControlNo') # df1 LEFT JOIN df2
    df_combine=df_combine.merge(df3, how='left', on='fac_no') # res LEFT JOIN df3
    df_combine=df_combine.merge(df4, how='left', on='fac_no') # res LEFT JOIN df4

    df_combine=df_combine.drop(columns=['ControlNo'])
    df_combine['code_no']=code_no

    return df_combine

In [5]:
df_node=pd.read_csv('temp.csv')
df_node_A1=df_node[df_node['fac_type']=='A1'] # 30 rows x 4 columns

# create an empty dataframe
df=pd.DataFrame()

for index, row in df_node_A1.iterrows():
    df_combine=query_data(row['fac_no'], row['code_no']) # by specifying 管制編號 and 代碼
    df=pd.concat([df, df_combine], ignore_index=True) # reset the index by setting the ignore_index to True

df.to_csv('ML format.csv', index=False)

<ipython-input-4-be6038134e92>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1=pd.read_sql(qstr.format(fac_no, code_no), conn)
<ipython-input-4-be6038134e92>:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2=pd.read_sql(qstr.format(fac_no), conn)
<ipython-input-4-be6038134e92>:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3=pd.read_sql(qstr.format(fac_no, code_no, voc_no, produce_code), conn) # garbled text because the type declared in DB is 'varchar' instead of 'nvarchar'
<ipython-input-4-be6038134e92>:34: U

In [6]:
df

,fac_no,AppType,DType,AppTime,ProduceCode,app_qty,AccName,VocCode1,Square1,SquareUnit1,Square2,SquareUnit2,Square3,SquareUnit3,VocCode1Name,Waste_name,ProduceName,Pd_qty_mMax,Pd_qty_mAvg,code_no
0,J59A7436,Store,3,2016/04,000000,0.00,力成科技股份有限公司3C廠,2613,16000.0,1,46271.32,1,74031.36,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,1.5,0.75,R-0201
1,J59A7436,Store,3,2016/05,000000,0.00,力成科技股份有限公司3C廠,2613,16000.0,1,46271.32,1,74031.36,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,1.5,0.75,R-0201
2,J59A7436,Store,3,2016/06,000000,0.00,力成科技股份有限公司3C廠,2613,16000.0,1,46271.32,1,74031.36,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,1.5,0.75,R-0201
3,J59A7436,Store,3,2016/07,000000,0.00,力成科技股份有限公司3C廠,2613,16000.0,1,46271.32,1,74031.36,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,1.5,0.75,R-0201
4,J59A7436,Store,3,2016/08,000000,0.00,力成科技股份有限公司3C廠,2613,16000.0,1,46271.32,1,74031.36,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,1.5,0.75,R-0201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,H43A0700,manu,3,2022/10,000000,10.00,日月光半導體製造股份有限公司中壢分公司封測五廠,2613,NaN,1,NaN,1,NaN,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,10.0,8.00,R-0201
3878,H43A0700,manu,3,2022/11,000000,10.00,日月光半導體製造股份有限公司中壢分公司封測五廠,2613,NaN,1,NaN,1,NaN,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,10.0,8.00,R-0201
3879,H43A0700,Store,3,2022/11,000000,0.89,日月光半導體製造股份有限公司中壢分公司封測五廠,2613,NaN,1,NaN,1,NaN,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,10.0,8.00,R-0201
3880,H43A0700,Store,3,2022/12,000000,0.00,日月光半導體製造股份有限公司中壢分公司封測五廠,2613,NaN,1,NaN,1,NaN,1,半導體封裝及測試業,廢塑膠,非製造程序產出類別,10.0,8.00,R-0201
